In [2]:
import pandas as pd
import os
from datetime import datetime

Iterate through the raw files and create a master file:

In [3]:
directory = os.getcwd() + "/spreadsheets/"
df2 = pd.DataFrame()
for filename in os.listdir(directory):
     if filename.endswith(".xlsx"):
        df = pd.read_excel(directory + filename)
        df = df[['   Rank', 'Emp.No.','Org. Unit', 'SEX', 'Race', 'Nationality', ' Marital St.', 'Certificate','Date of Birth', 'Date of Enlistment']]
        date = datetime.strptime(filename[0:-5], '%b %y')
        df['Date of Headcount'] = date.date()
        df2 = df2.append(df)
df2.to_csv("df2.csv",index=False)

In [13]:
df2.rename(columns={'   Rank':'Rank', 'Emp.No.':'Staff ID', ' Marital St.':'Marital Status'}, inplace=True)
df2['Date of Headcount'] = pd.to_datetime(df2['Date of Headcount'])
df2['Date of Headcount'] = df2['Date of Headcount'] + pd.offsets.MonthEnd(n=1) #Headcount is at month end

In [14]:
df2 = df2[df2['Org. Unit']!='APF HQ Recruits Branch']
df2 = df2[df2['Org. Unit']!='Planning']
df2 = df2[df2['Org. Unit']!='Manpower Planning Officer']

Merge education rank:

In [15]:
education = pd.read_csv("educationrank.csv")
master = pd.merge(df2, education, how="left", left_on='Certificate', right_on='Certificate text')
master.rename(columns={'SEX':'Sex', 'Ranking':'Education Rank'}, inplace=True)
master.drop(['Certificate','Certificate text'], axis=1, inplace=True)

Merge org unit groups:

In [16]:
orgunits = pd.read_csv("orgunits.csv")
master = pd.merge(master, orgunits, how="left", on="Org. Unit")
master.rename(columns={'Org. Unit Grouping':'Org Unit Group'}, inplace=True)
master.drop(['Org. Unit'], axis=1, inplace=True)

In [18]:
#master['Length of Service'] = master['Date of Headcount'] - master['Date of Enlistment']
#master['Age'] = master['Date of Headcount'] - master['Date of Birth']
master.to_csv("master.csv", index=False)

In [17]:
master.head()

,Rank,Staff ID,Sex,Race,Nationality,Marital Status,Date of Birth,Date of Enlistment,Date of Headcount,Education Rank,Org Unit Group
0,Corporal,5183,MALE,Indian,Singaporean,Marr.,1952-02-16,1973-04-02,2018-01-31,11.0,Inspectorate
1,Corporal,5291,MALE,Indian,Singaporean,Marr.,1950-04-13,1973-12-10,2018-01-31,10.0,AVA Fisheries
2,Sergeant,5309,MALE,Malay,Singaporean,Marr.,1953-09-29,1973-12-10,2018-01-31,11.0,Custom
3,Lance Corporal,5468,MALE,Indian,Singaporean,Marr.,1950-12-06,1970-06-01,2018-01-31,8.0,SGH (Ward Patrol) & Rider
4,Lance Corporal,5714,MALE,Malay,Singaporean,Marr.,1948-02-12,1967-02-01,2018-01-31,8.0,Sector P1
